### Deployed in Colabs with TPU accelerator

In [12]:
# import the necessary libraries to execute this code
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.model_selection import RandomizedSearchCV as RSCV

# build NN for class
from tensorflow.keras.layers import  Dropout
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Dense,Dropout
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor


ModuleNotFoundError: No module named 'scikeras'

# Data Processing

In [7]:

df=pd.read_excel('/kaggle/input/cfs-plasma-proteins/Plasma_for_CFS.xlsx')
inputDf=df.iloc[55:112,2:1348]

titles=np.concatenate((df.iloc[54,2:31],df.iloc[45,31:1348]),axis=None)
inputDf.columns=titles

inputDf=inputDf[inputDf['SampleType']=="Sample"]
inputDf.index=range(0,len(inputDf.index)) # update row numbers

yArr=(inputDf["SampleGroup"]=="Patient").astype(int) # patient is 1, control is 0

xArr=pd.concat([(inputDf["Gender"]=="F").astype(int),inputDf["Age"],inputDf.iloc[0:57,29:1346]], axis=1)
featureTitles=xArr.columns

display(xArr)

,Gender,Age,CHIP,CEBPB,NSE,PIAS4,IL-10 Ra,STAT3,IRF1,c-Jun,...,UB2G2,Transgelin-2,ATPO,Corticotropin-lipotropin,QORL1,PEDF,CATF,FTCD,UBP25,PLXB2
0,1,25,596.8,1277.7,622.1,689.1,1825.1,415.8,791.3,2539.1,...,6233.7,3864.8,1474.8,952.1,1923.5,38280.5,3298.5,8208.4,1931.4,4597.4
1,0,43,673.8,1932.8,653.2,836.2,1752.5,560.6,719.1,3830.2,...,6218.3,3896,4185.5,1703.5,1791.1,50142.8,3527.2,8245.3,2095.9,3399.3
2,1,24,722.3,1509.8,643.2,923.5,1752.5,610.3,752,2319.8,...,5335.7,3877.8,1542.9,1193.8,1912.5,35329.8,2922.3,1353.8,1669.3,3739.6
3,1,51,583.4,2124,630.2,1126.3,1590.7,825.3,775,2487.9,...,5763.3,4219.6,3029.7,775.2,2642.1,39961.3,5114.1,3677.9,1521.1,4053.5
4,0,37,790.8,1859,650.6,1028.1,1797.3,772.9,780.8,3082.4,...,5904,4369.3,2461.9,2201.7,1532.6,43351.6,3077.7,3900.4,1914.8,3839.1
5,0,22,901.3,1401.9,643.7,784.6,1670.4,508.5,687.4,3015.2,...,5166.2,4737,2044.6,1563.6,1509.8,48178.9,2332.4,4078.9,1759.5,4002
6,1,56,616.5,1465.9,681.5,979.7,1731.1,519.4,717.2,2656.5,...,7101.6,5593.5,1519.9,1176.6,1913.8,53541.1,3440.3,26390.8,1816.2,2690.2
7,1,26,786.2,1809.6,554.9,1036.8,1659.6,569.9,840.3,2841.1,...,6314,4706.9,1778.7,1063.1,1612.1,37400.9,3420.5,3754.2,1711.2,3284.4
8,1,22,828.6,1769.9,624.6,976.2,1568.5,963.4,757.1,2587.9,...,4841.9,5338.7,1962.1,901.8,1781.2,43965.5,2870.2,1707.5,1923.1,2937.5
9,0,60,1271.4,1529.7,687,914.6,1651.5,450.4,1078.2,2718.7,...,5716.6,5073.2,1827.4,1158,1750.5,43568.1,4230.4,4197.8,1666.7,4105.1


In [8]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# instantiate a distribution strategy
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

ValueError: Please provide a TPU Name to connect to.

In [21]:
from scikeras.wrappers import KerasRegressor

# create a function that will build and compile a Keras model

def NN_builder(n_hidden=1, optimizer = 'rmsprop', units=40, learning_rate = 0.001, input_shape=[17], 
               regularization=0.001, dropout=0.2, activation = 'sigmoid'):
    #with tpu_strategy.scope():
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=input_shape))

        for layer in range (n_hidden):
            model.add(keras.layers.Dense(units=40, activation=activation, activity_regularizer=l1_l2(regularization)))
            model.add(Dropout(dropout))

        model.add(keras.layers.Dense(units=1,activation='sigmoid'))
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

        model.compile(loss="mean_absolute_error", optimizer=optimizer)
        return model

#NN = KerasRegressor(model=NN_builder,n_hidden=1, optimizer = 'rmsprop', units=40, learning_rate = 0.001, input_shape=[17], 
               #regularization=0.001, dropout=0.2, activation = 'sigmoid')
NN = KerasRegressor(NN_builder)

ImportError: cannot import name '_fit_context' from 'sklearn.base' (/opt/conda/lib/python3.10/site-packages/sklearn/base.py)

In [19]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 72.2 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [10]:
model = NN
p_grid ={"n_hidden" : [1,2,3,4,5],
         "units" : [10,20,30,40,50],
         "learning_rate": [0.0001,0.001,0.01],
         "regularization":[1e-2,1e-3,1e-4],
         "dropout":[0.0,0.1,0.2,0.3],
         "batch_size":[32, 64, 128, 256],
         "activation": ['relu', 'tanh', 'sigmoid']}

xArr.columns = xArr.columns.astype(str)
stdScale=StandardScaler().fit(xArr)
X=stdScale.transform(xArr)
Y = yArr.astype('int')

AttributeError: module 'tensorflow.keras' has no attribute 'wrappers'

In [ ]:
from tensorflow.keras.layers import  Dropout
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Dense,Dropout
import tensorflow as tf
from tensorflow import keras

# Number of epochs
epochs = 50

# Define callaback
callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=5)]

NUM_TRIALS = 10

itr_number = [] # create new empty list for itr number 
outer_results = []
inner_results = []
model_params = []
y_test_list = []
pred_list = []
pred_var_list = []

for i in range(NUM_TRIALS): #configure the cross-validation procedure - outer loop (test set) 

      cv_outer = ShuffleSplit(n_splits=1, test_size=0.2, random_state=i) #hold back 20% of the groups for test set

      # split data using GSS
      for train_index, test_index in cv_outer.split(X, Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # store test set information
        y_test = np.array(y_test) #prevents index from being brought from dataframe
        y_test_list.append(y_test)

        # configure the cross-validation procedure - inner loop (validation set/HP optimization)
        cv_inner = KFold(n_splits=10) #should be 10 fold group split for inner loop

        # define search space
        search = RSCV(model, p_grid, n_iter=50, verbose=3, scoring='neg_mean_absolute_error', cv=cv_inner, refit=True) # should be 100

        # execute search
        result = search.fit(X_train, y_train, callbacks=callbacks, epochs=epochs, verbose=0)

        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_

        # get the score for the best performing model and store
        best_score = abs(result.best_score_)
        inner_results.append(best_score)

        # evaluate model and estimate epistemic uncertainty on the hold out dataset
        predictions = []
        for _ in range(100):
            predictions += [best_model.predict(X_test, verbose=0)]
        
        yhat, pred_unbiased = np.mean(np.array(predictions), axis=0), np.std(np.array(predictions), axis=0)

        # store drug release predictions
        pred_list.append(yhat)
            
        # store prediction variance
        pred_var_list.append(pred_unbiased)

        # evaluate the model
        acc = mean_absolute_error(y_test, yhat)

        # store the result
        itr_number.append(i+1)
        outer_results.append(acc)
        model_params.append(result.best_params_)

      # report progress at end of each inner loop
      print('\n################################################################\n\nSTATUS REPORT:') 
      print('Iteration '+str(i+1)+' of '+str(NUM_TRIALS)+' runs completed') 
      print('Test_Score: %.3f, Best_Valid_Score: %.3f, \n\nBest_Model_Params: \n%s' % (acc, best_score, result.best_params_))
      print("\n################################################################\n ")


In [1]:
#create dataframe with results of nested CV
list_of_tuples = list(zip(itr_number, inner_results, outer_results, model_params, G_test_list, E_test_list, T_test_list, y_test_list, pred_list, pred_var_list))
CV_dataset = pd.DataFrame(list_of_tuples, columns = ['Iter', 'Valid Score', 'Test Score', 'Model Parms', 'DP_Groups', "Experimental Index", "Time", 'Experimental_Release', 'Predicted_Release','Prediction_Variance'])
CV_dataset['Score_difference'] = abs(CV_dataset['Valid Score'] - CV_dataset['Test Score']) #Groupby dataframe model iterations that best fit the data (i.e., validitaion <= test)
CV_dataset.sort_values(by=['Score_difference', 'Test Score'], ascending=True, inplace=True) 
CV_dataset = CV_dataset.reset_index(drop=True) # Reset index of dataframe
CV_dataset.to_pickle("/kaggle/working/NN.pkl") # save dataframe as pickle file


NameError: name 'itr_number' is not defined

In [ ]:
import pickle
# assign the best model paramaters
best_model_params = CV_dataset.iloc[0,3]
# set params from the best model to a class object
best_model = model.set_params(**best_model_params)
best_model = best_model.fit(X, Y)
with open('/kaggle/working/NN_model.pkl', 'wb') as file: # Save the Model to pickle file
          pickle.dump(best_model, file)

In [ ]:
CV_dataset.describe()